In [1]:
import mlrun, os
# for local DB path use 'User/mlrun' instead 
mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

In [2]:
out_path='/User/demo-xgb-project/artifacts'
os.makedirs(out_path, exist_ok=True)

### get data

In [3]:
acquire_fn = mlrun.import_function('/User/functions-refac/get_toy_data/function.yaml').apply(mlrun.mount_v3io()) 

task = mlrun.NewTask(params={'dataset': 'iris'}, out_path=out_path )

acquire_fn_run = acquire_fn.run(task)

[mlrun] 2020-03-12 20:12:14,727 starting run get-toy-data-get_toy_data uid=790d67aa36e24692856790d610e6413c  -> http://mlrun-api:8080
[mlrun] 2020-03-12 20:12:14,807 Job is running in the background, pod: get-toy-data-get-toy-data-jm2kn
[mlrun] 2020-03-12 20:12:19,245 log artifact iris at /User/demo-xgb-project/artifacts/iris.pqt, size: None, db: Y

[mlrun] 2020-03-12 20:12:19,254 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...e6413c,0,Mar 12 20:12:19,completed,get-toy-data-get_toy_data,host=get-toy-data-get-toy-data-jm2knkind=jobowner=admin,,dataset=iris,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run 790d67aa36e24692856790d610e6413c  , !mlrun logs 790d67aa36e24692856790d610e6413c 
[mlrun] 2020-03-12 20:12:21,078 run executed, status=completed


In [4]:
acquire_fn_run.outputs

{'iris': '/User/demo-xgb-project/artifacts/iris.pqt'}

### describe

In [5]:
describe_fn = mlrun.import_function('/User/functions-refac/describe/function.yaml').apply(mlrun.mount_v3io()) 

task = mlrun.NewTask(
    "sum",
    params={"key": "summary", "label_column": "labels"},
    inputs={"table": acquire_fn_run.outputs['iris']},
    out_path=out_path )

describe_fn_run = describe_fn.run(task)

[mlrun] 2020-03-12 20:12:21,139 starting run sum uid=88733c25b1e1440a9d85f571d9dc5421  -> http://mlrun-api:8080
[mlrun] 2020-03-12 20:12:21,225 Job is running in the background, pod: sum-5knrr
Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
main.py:95: UserWarning: To output multiple subplots, the figure containing the passed axes is being cleared
  figarray = table.hist(ax=ax, ylabelsize=5, xlabelsize=5)
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
[mlrun] 2020-03-12 20:12:26,355 log artifact summary at /User/demo-xgb-project/artifacts/summary.csv, size: None, db: Y
[mlrun] 2020-03-12 20:12:26,423 log artifact scale_pos_weight at /User

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...dc5421,0,Mar 12 20:12:26,completed,sum,host=sum-5knrrkind=jobowner=admin,table,key=summarylabel_column=labels,,summaryscale_pos_weightimbalancecorrelationhistograms


to track results use .show() or .logs() or in CLI: 
!mlrun get run 88733c25b1e1440a9d85f571d9dc5421  , !mlrun logs 88733c25b1e1440a9d85f571d9dc5421 
[mlrun] 2020-03-12 20:12:30,371 run executed, status=completed


In [6]:
describe_fn_run.outputs

{'summary': '/User/demo-xgb-project/artifacts/summary.csv',
 'scale_pos_weight': '/User/demo-xgb-project/artifacts/scale_pos_weight',
 'imbalance': '/User/demo-xgb-project/artifacts/plots/imbalance.html',
 'correlation': '/User/demo-xgb-project/artifacts/plots/corr.html',
 'histograms': '/User/demo-xgb-project/artifacts/plots/hist.html'}

### get model

In [7]:
get_model_config_fn = mlrun.import_function('/User/functions-refac/get_model_config/function.yaml').apply(mlrun.mount_v3io()) 

task = mlrun.NewTask(params={'config_url': '/User/demo-xgb-project/xgboost-conf.json'}, out_path=out_path )

get_model_config_fn_run = get_model_config_fn.run(task)

[mlrun] 2020-03-12 20:12:30,398 starting run get-model-config-get_model_config uid=fca9e4621b9848a2ba8f824b63243cc5  -> http://mlrun-api:8080
[mlrun] 2020-03-12 20:12:30,701 Job is running in the background, pod: get-model-config-get-model-config-2jsbl
[mlrun] 2020-03-12 20:12:34,793 log artifact class_params at /User/demo-xgb-project/artifacts/class_params.json, size: 377, db: Y
[mlrun] 2020-03-12 20:12:34,805 log artifact fit_params at /User/demo-xgb-project/artifacts/fit_params.json, size: 69, db: Y

[mlrun] 2020-03-12 20:12:34,814 run executed, status=completed
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...243cc5,0,Mar 12 20:12:34,completed,get-model-config-get_model_config,host=get-model-config-get-model-config-2jsblkind=jobowner=admin,,config_url=/User/demo-xgb-project/xgboost-conf.json,,class_paramsfit_params


to track results use .show() or .logs() or in CLI: 
!mlrun get run fca9e4621b9848a2ba8f824b63243cc5  , !mlrun logs fca9e4621b9848a2ba8f824b63243cc5 
[mlrun] 2020-03-12 20:12:36,901 run executed, status=completed


In [8]:
get_model_config_fn_run.outputs

{'class_params': '/User/demo-xgb-project/artifacts/class_params.json',
 'fit_params': '/User/demo-xgb-project/artifacts/fit_params.json'}

### train model

In [12]:
train_fn = mlrun.import_function('/User/functions-refac/sklearn_classifier/function.yaml').apply(mlrun.mount_v3io()) 

task = mlrun.NewTask(
    params={'sample' : -1,
            'label_column' : 'labels'}, 
    inputs={'data_key'     : acquire_fn_run.outputs['iris'],
            'class_params' : get_model_config_fn_run.outputs['class_params'],
            'fit_params'   : get_model_config_fn_run.outputs['fit_params'],},
    out_path=out_path )

train_model_run = train_fn.run(task)

[mlrun] 2020-03-12 20:13:25,626 starting run sklearn-classifier-train_model uid=8ed309ad741f48a79e406adbfbb2c7e9  -> http://mlrun-api:8080
[mlrun] 2020-03-12 20:13:25,744 Job is running in the background, pod: sklearn-classifier-train-model-bhpqp
Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html
No handles with labels found to put in legend.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
[mlrun] 2020-03-12 20:13:31,038 log artifact test-set at /User/demo-xgb-project/artifacts/test-set.csv, size: None, db: Y
[mlrun] 2020-03-12 20:13:31,127 log artifact model at /User/demo-xgb-project/artifacts/models/model.pkl, size: None, db: Y
[mlrun] 2020-03-12 20:13:31,321 log artifact roc at /User/demo-xgb-project/artifacts/plot

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...b2c7e9,0,Mar 12 20:13:30,completed,sklearn-classifier-train_model,host=sklearn-classifier-train-model-bhpqpkind=jobowner=admin,class_paramsdata_keyfit_params,label_column=labelssample=-1,accuracy=0.9117647058823529avg_precscore=0.9676813220337889f1_score=0.9117647058823528rocauc=0.9678194028194028,test-setmodelrocconfusionfeatimp.pngfeatimp.csv


to track results use .show() or .logs() or in CLI: 
!mlrun get run 8ed309ad741f48a79e406adbfbb2c7e9  , !mlrun logs 8ed309ad741f48a79e406adbfbb2c7e9 
[mlrun] 2020-03-12 20:13:34,932 run executed, status=completed


In [14]:
train_model_run.outputs

{'accuracy': 0.9117647058823529,
 'avg_precscore': 0.9676813220337889,
 'f1_score': 0.9117647058823528,
 'rocauc': 0.9678194028194028,
 'test-set': '/User/demo-xgb-project/artifacts/test-set.csv',
 'model': '/User/demo-xgb-project/artifacts/models/model.pkl',
 'roc': '/User/demo-xgb-project/artifacts/plots/roc.png',
 'confusion': '/User/demo-xgb-project/artifacts/plots/confusion.png',
 'featimp.png': '/User/demo-xgb-project/artifacts/plots/featimp.png',
 'featimp.csv': '/User/demo-xgb-project/artifacts/featimp.csv'}

<a id="pipeline"></a>
______________________________________________
## Create a multi-stage KubeFlow Pipeline from our functions
* Load Iris dataset into a CSV
* Train a model using XGBoost with Hyper-parameter
* Deploy the model using Nuclio-serving
* Generate a plot of the training results

In [15]:
import kfp
from kfp import dsl

**create a model serving function from the [model-serving notebook](nuclio-serving.ipynb)** 

This function will be used in our workflow

In [24]:
# define a nuclio-serving functions, generated from a remote notebook file
srvfn = mlrun.new_model_server('iris-serving', 
                         model_class='XGBoostModel', 
                         filename='https://raw.githubusercontent.com/mlrun/functions/master/serving/xgboost/xgb_serving.ipynb')

# attach to the fabric (to read the model file)
srvfn.apply(mlrun.mount_v3io())

**define a 4 step workflow with hyper-params**

In [25]:
@dsl.pipeline(
    name='My XGBoost training pipeline',
    description='Shows how to use mlrun.'
)
def xgb_pipeline(
   #eta = [0.1, 0.2, 0.3], gamma = [0.1, 0.2, 0.3]
):
    ingest = acquire_fn.as_step(
        name='acquire',
        params={'dataset': 'iris'},
        outputs=['data'],
        out_path=ARTIFACT_PATH)

    describe = describe_fn.as_step(
        name='describe',
        params={"key": "summary", "label_column": "labels"},
        inputs={"table": ingest.outputs['data']},
        outputs=['data'],
        out_path=ARTIFACT_PATH)
    
    configs = get_model_config_fn.as_step(
        name='config',
        params={'config_url': '/User/demo-xgb-project/xgboost-conf.json'},
        outputs=['class_params', 'fit_params'],
        out_path=ARTIFACT_PATH)
    
    train = train_fn.as_step(
        name='train',
        params={'sample': -1, 'label_column': 'labels'},
        inputs={'data_key'    : ingest.outputs['data'],
                'class_params': configs.outputs['class_params'],
                'fit_params'  : configs.outputs['fit_params']},
        outputs=['model'],
        out_path=ARTIFACT_PATH)

 
    # deploy the model serving function with inputs from the training stage
    deploy = srvfn.deploy_step(project = 'iris', models={'iris_v1': train.outputs['model']})

#### Create a KubeFlow client and submit the pipeline with parameters

**define the artifacts output path**
the pipeline outputs will be writtento the artifacts path directory, the path can be a file path (require volume mounts) or an object path (v3io://, s3://, ..).

if we specify `{{workflow.uid}}` in the path it will be replaced with the actual workflow ID, this way every workflow run will store artifacts in a unique location for reproducability.

In [26]:
from mlrun import run_pipeline
ARTIFACT_PATH = 'v3io:///users/admin/demo-xgb-project/artifacts/{{workflow.uid}}/'
arguments = {}
#arguments = {'eta': [0.05, 0.10, 0.40, 0.5], 'gamma': [0.1, 0.3, 0.6]}

In [27]:
id = run_pipeline(xgb_pipeline, arguments, experiment='xgb', artifact_path=ARTIFACT_PATH)

[mlrun] 2020-03-12 20:17:52,814 Pipeline run id=93291f77-c033-4252-ba97-21d5f87dc75f, check UI or DB for progress


### check the resilts of our pipeline

In [28]:
# connect to the run db 
db = mlrun.get_run_db().connect()

In [29]:
# query the DB with filter on workflow ID (only show this workflow) 
db.list_runs('', labels=f'workflow={id}').show()

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts


**[back to top](#top)**